# 課題 1.1　線形回帰分析

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

autos = pd.read_csv('data/imports-85.data', na_values='?')
autos.columns = ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration',
                    'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
                    'wheel_base', 'length','width', 'height', 'curb_weight',
                    'engine_type', 'num_of_cylinders', 'engine_size', 'fuel_system',
                    'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm',
                    'city_mpg', 'highway_mpg','price']
# 表示する最大列数の設定
pd.set_option('display.max_columns', len(autos.columns))
# 欠損値 NaN がある行の除去
autos = autos.dropna()
# 確認
autos

In [ ]:
# price 以外の量的データのラベルを昇順にソート
qd_labels = autos.describe().columns.drop('price').sort_values()
# 確認
qd_labels

<hr>

### 1.1.1 VIFが5以上の説明変数を除外

price 以外の量的データである説明変数について、昇順にソートしたラベル qd_labels の先頭からVIFが5以上の変数を除外し、残った説明変数について、説明変数とVIFを行とする pandas DataFrame を作成し、表示しなさい。
- 結果は、行ラベルは説明変数、列ラベルはVIFのn行1列の DataFrame

### VIF

![VIF](slides/4_42.png)

$x_{i}$ を $x_{i}$ 以外の説明変数で予測するモデルを作成
- $x_{i}$ が目的変数
- $x_{i}$ 以外が説明変数

決定係数 $R^{2}$
- $R$ の2乗ではないことに注意

### 解釈1

全ての変数について、1つを目的変数、残りを説明変数として、VIFを計算する。<br>
VIFが5以上となった変数は除去する。

- 例えば city_mpg, highway_mpg は高い相関を持つため、両方とも除去されてしまう。

In [ ]:
model = LinearRegression()

# 線形回帰モデルを作成、決定係数を返す関数を定義
# - Y_label を目的変数
# - Y_label 以外を説明変数
def LR_score(df, Y_label):
    # 説明変数、目的変数
    X = df.drop([Y_label], axis=1)
    Y = df[Y_label]
    # 学習、予測値
    model.fit(X, Y)
    Y_predict = model.predict(X)
    # 決定係数
    return r2_score(Y, Y_predict)

# VIFを計算する関数を定義
# - df: 対象とする全ての変数が入った DataFrame
def calc_vif(df):
    # VIF を入れる DataFrame の初期化
    df_vif = pd.DataFrame(index=qd_labels, columns=['vif'])

    # df.columns（全ての変数ラベル）から、順番に1つずつ取り出す
    # 取り出したラベル Y_label を LR_score() の引数として呼び出す
    # - Y_label を目的変数とした決定係数が戻り値
    for Y_label in df.columns:
        # VIF の計算
        _vif = 1 / (1 - LR_score(df, Y_label))
        # 確認用に目的変数ごとのVIFを出力
        print(Y_label, _vif)
        # DataFrame への代入
        df_vif.loc[Y_label] = _vif
    return df_vif

# 量的データ
df = autos[qd_labels]
# VIFの計算
df_vif = calc_vif(df)
# VIFが5以上の変数を除外 = VIF < 5 のみを残す
df_vif = df_vif.query('vif < 5')
# 確認
df_vif

- 結果は、行ラベルは説明変数、列ラベルはVIFのn行1列の DataFrame

<hr>

### 1.1.2 決定係数

すべての質的データの説明変数と1.1.1で残った説明変数を合わせて説明変数、price を目的変数として重回帰モデルを作成し、そのモデルの決定係数を求めなさい。<br>
小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること（0.7625ならば0.763が答え）。

In [ ]:
# qd_labels の列をdropして、質的データと price を残した列ラベル
# - price を入れるのは LR_score() で決定係数を計算しようとしているため
X_labels = autos.columns.drop(qd_labels)

# 確認
X_labels

In [ ]:
# df_vif に残った説明変数を追加した列ラベル
X_labels = X_labels.append(df_vif.index)

# 確認
X_labels

In [ ]:
# 編集した列ラベルからなる DataFrame
df = autos[X_labels]

# ダミー変数化
df = pd.get_dummies(data=df, drop_first=True)

# 決定係数の計算
# - price を目的変数
R2 = LR_score(df, 'price')
print('R2={:.3f}'.format(R2))
# 四捨五入されることの確認用
print('R2={:.5f}'.format(R2))

<hr>

### 1.1.3 回帰係数

1.1.2で作成したモデルの回帰係数を、説明変数と回帰係数を行とする pandas DataFrame を作成し、降順で表示しなさい（行数が多い場合、途中の表示が省略されないようにすること）。

In [ ]:
# 表示行数の設定
pd.set_option('display.max_columns', len(df.columns))

# 回帰係数の DataFrame を作成
# - index: 行ラベルの指定
# - columns: 列ラベルの指定
df_coef = pd.DataFrame(model.coef_, index=df.columns.drop('price'), columns=['coefficient'])

# sort_values：ソート
# - by='coefficient' 回帰係数でソート
# - ascending=False 降順を指定
df_coef.sort_values(by='coefficient', ascending=False)